In [1]:
import pandas as pd
import datetime
import numpy as np
import os

def recursive_file_gen(my_root_dir):
    for root, dirs, files in os.walk(my_root_dir):
        for file in files:
            yield os.path.join(root, file)

In [2]:
list_1_after_201806=[x for x in list(recursive_file_gen("/home/jian/BigLots/2018_by_weeks/")) if ("aily" in x) & (".txt" in x) ]
folder_date=[datetime.datetime.strptime(x.split("/")[len(x.split("/"))-2].split("_")[1],"%Y-%m-%d").date() for x in list_1_after_201806]
df_1_after_201806=pd.DataFrame({"date":folder_date,"path":list_1_after_201806},index=[x for x in range(len(list_1_after_201806))])
df_1_after_201806['date'].apply(lambda x: x.weekday()).unique()
df_1_after_201806=df_1_after_201806.sort_values("date").reset_index()
del df_1_after_201806['index']

In [3]:
list_2_before_201806=list(recursive_file_gen("/home/jian/BigLots/hist_daily_data/"))
folder_date=[datetime.datetime.strptime(x.split("/")[len(x.split("/"))-1].split("_")[3].split(".")[0],"%Y-%m-%d").date() for x in list_2_before_201806]
df_2_before_201806=pd.DataFrame({"date":folder_date,"path":list_2_before_201806},index=[x for x in range(len(list_2_before_201806))])
df_2_before_201806['date'].apply(lambda x: x.weekday()).unique()
df_2_before_201806=df_2_before_201806.sort_values('date').reset_index()
del df_2_before_201806['index']
df_2_before_201806['date'].apply(lambda x: x.weekday()).unique()

array([5])

In [4]:
date_2017_Q3=[datetime.date(2017,8,12)+datetime.timedelta(days=x*7) for x in range(13)]
date_2018_Q3=[datetime.date(2018,8,11)+datetime.timedelta(days=x*7) for x in range(13)]

df_1_after_201806_Q3=df_1_after_201806[df_1_after_201806['date'].isin(date_2018_Q3)]
df_2_before_201806_Q3=df_2_before_201806[df_2_before_201806['date'].isin(date_2017_Q3)]
df_all_Q3_txt=df_1_after_201806_Q3.append(df_2_before_201806_Q3)

In [5]:
mattress_class_ids=[str(x) for x in range(61501,61506)]
print(mattress_class_ids)

def func_week_end_date(x):
    if x.weekday()==6:
        y=x+datetime.timedelta(days=6)
    else:
        y=x+datetime.timedelta(days=5-x.weekday())
    return y

['61501', '61502', '61503', '61504', '61505']


In [6]:
agg_mattress_all_stores=pd.DataFrame()
count_i=0
for file_path in df_all_Q3_txt['path'].unique().tolist():
    df=pd.read_table(file_path,dtype=str,sep="|")
    
    df['transaction_dt']=df['transaction_dt'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").date())
    week_end_date=df['transaction_dt'].max()
    
    df['rewards_label']=np.where(pd.isnull(df['customer_id_hashed']),"Non_Rewards","Rewards")
    df['subclass_transaction_amt']=df['subclass_transaction_amt'].astype(float)
    df_mattress=df[df['class_code_id'].isin(mattress_class_ids)]
    df_mattress['week_end_date']=df_mattress['transaction_dt'].apply(lambda x: func_week_end_date(x))
    if df_mattress['week_end_date'].unique().tolist()==[week_end_date]:
        df_mattress_sales=df_mattress.groupby(['location_id','rewards_label','week_end_date'])['subclass_transaction_amt'].sum().to_frame().reset_index()
        
        df_mattress_sales_rewards=df_mattress_sales[df_mattress_sales['rewards_label']=="Rewards"]
        df_mattress_sales_rewards=df_mattress_sales_rewards.rename(columns={"subclass_transaction_amt":"rewards_mattress_sales"})
        del df_mattress_sales_rewards['rewards_label']
        df_mattress_sales_Nonrewards=df_mattress_sales[df_mattress_sales['rewards_label']=="Non_Rewards"]
        df_mattress_sales_Nonrewards=df_mattress_sales_Nonrewards.rename(columns={"subclass_transaction_amt":"Non_rewards_mattress_sales"})
        del df_mattress_sales_Nonrewards['rewards_label']
        df_output=pd.merge(df_mattress_sales_rewards,df_mattress_sales_Nonrewards,on=['location_id','week_end_date'],how="outer")
        df_output=df_output.fillna(0)
        # 
        df_mattress=df_mattress[df_mattress['subclass_transaction_amt']>=0]
        df_mattress_trans=df_mattress.groupby(['location_id','rewards_label','week_end_date'])['subclass_transaction_amt'].count().to_frame().reset_index()
        
        df_mattress_trans_rewards=df_mattress_trans[df_mattress_trans['rewards_label']=="Rewards"]
        df_mattress_trans_rewards=df_mattress_trans_rewards.rename(columns={"subclass_transaction_amt":"rewards_mattress_trans"})
        del df_mattress_trans_rewards['rewards_label']
        df_mattress_trans_Nonrewards=df_mattress_trans[df_mattress_trans['rewards_label']=="Non_Rewards"]
        df_mattress_trans_Nonrewards=df_mattress_trans_Nonrewards.rename(columns={"subclass_transaction_amt":"Non_rewards_mattress_trans"})
        del df_mattress_trans_Nonrewards['rewards_label']
        df_output=pd.merge(df_mattress_sales_rewards,df_mattress_sales_Nonrewards,on=['location_id','week_end_date'],how="outer")
        df_output=pd.merge(df_output,df_mattress_trans_rewards,on=['location_id','week_end_date'],how="outer")
        df_output=pd.merge(df_output,df_mattress_trans_Nonrewards,on=['location_id','week_end_date'],how="outer")
        
        df_output=df_output.fillna(0)
        agg_mattress_all_stores=agg_mattress_all_stores.append(df_output)
        count_i+=1
        print(count_i,datetime.datetime.now())
    else:
        print("Date Error: ",week_end_date)
        
    

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


1 2018-12-17 16:33:37.177675
2 2018-12-17 16:36:19.277390
3 2018-12-17 16:39:07.360461
4 2018-12-17 16:41:54.763943
5 2018-12-17 16:44:48.232750
6 2018-12-17 16:47:32.338697
7 2018-12-17 16:50:04.205717
8 2018-12-17 16:53:16.461945
9 2018-12-17 16:56:54.842336
10 2018-12-17 16:59:28.589125
11 2018-12-17 17:02:05.889094
12 2018-12-17 17:04:55.715139
13 2018-12-17 17:07:51.950017
14 2018-12-17 17:10:54.385906
15 2018-12-17 17:13:56.907480
16 2018-12-17 17:16:58.143463
17 2018-12-17 17:20:04.275334
18 2018-12-17 17:23:35.388383
19 2018-12-17 17:26:33.144070
20 2018-12-17 17:29:28.883015
21 2018-12-17 17:33:25.065118
22 2018-12-17 17:37:34.478112
23 2018-12-17 17:40:31.287971
24 2018-12-17 17:43:15.705396
25 2018-12-17 17:46:05.934967
26 2018-12-17 17:49:18.043142


In [10]:
agg_mattress_all_stores.to_csv('/home/jian/Projects/Big_Lots/Analysis/2018_Q3/Mattress_Since_Aug/mattress_5_class_codes_data_JL_'+str(datetime.datetime.now().date())+".csv",index=False)